# Interactive Maps Exploring Relationships Between Fermented Vegetables and Covid-19 Mortality Rates

The original [paper](https://www.medrxiv.org/content/10.1101/2020.07.06.20147025v1) suggests that low COVID-19 death rates at the country level were linked to high fermented vegetable consumption in Europe. However, this conclusion was based on data from June 2020, an early stage of the three-year pandemic. I aim to explore whether this finding holds with the latest data using interactive maps. Additionally, I will visualize longitudinal trends in death rates or absolute death numbers.

## Load modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import kaleido # This package is required to save the map as a static image

: 

## Load and preprocess epidemiological data

### Covid 19 mortality and population data

In [ ]:
# Load Covid-19 death data
covid_death_df = pd.read_csv('time_series_covid19_deaths_global.csv')

In [ ]:
covid_death_df.head() 

The Covid-19 death data contains geographic information in the first four columns, followed by daily death counts. I will aggregate the data to get the total number of deaths and death rates per country. 

### Aggregate yearly death counts for each country

In [ ]:
# Make wide table long 
covid_death_df_long = covid_death_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name='Date', value_name='Deaths') 

In [ ]:
covid_death_df_long.shape

In [ ]:
covid_death_df_long['Date'].head()

In [ ]:
# Add year column based on last two digits of Date column
covid_death_df_long['Date'] = pd.to_datetime(covid_death_df_long['Date'], format='%m/%d/%y', errors='coerce')


In [ ]:
covid_death_df_long['Date'].head()

In [ ]:
# Check missing values in Date column
covid_death_df_long['Date'].isnull().sum() 

In [ ]:
covid_death_df_long['Year'] = covid_death_df_long['Date'].dt.year

In [ ]:
# Aggregate deaths by country and year
covid_death_df_agg = covid_death_df_long.groupby(['Country/Region', 'Year'])['Deaths'].sum().reset_index()

In [ ]:
covid_death_df_agg.head()

In [ ]:
# Aggregate daily death counts to get total death count
covid_death_df['Total Deaths'] = covid_death_df.iloc[:, 4:].sum(axis=1) 

In [ ]:
covid_death_df['Total Deaths'].describe() 

In [ ]:
# Inspect Country/Region column
covid_death_df['Country/Region'].value_counts()

In [ ]:
# Inspect Province/State column
covid_death_df['Province/State'].value_counts()

There are multiple states or provinces within a country in the data. I will aggregate the data to the country level.

In [ ]:
# Aggregate total deaths by country 
covid_death_country_df = covid_death_df.groupby('Country/Region')['Total Deaths'].sum().reset_index() 

In [ ]:
# Sort countries by total deaths in descending order 
covid_death_country_df = covid_death_country_df.sort_values(by='Total Deaths', ascending=False) 

In [ ]:
covid_death_country_df.head()

In [ ]:
covid_death_country_df.tail()

In [ ]:
covid_death_country_df.rename(columns={'Country/Region': 'Country'}, inplace=True)

The dataframe `covid_death_country_df` contains aggregated COVID-19 deaths at the country level from January 2020 to March 2023, used for the following visualization.

### Population data for EU countries

In [ ]:
pop_df = pd.read_excel('demo_gind__custom_7680622_page_spreadsheet.xlsx', sheet_name='Sheet 1', skiprows=7)

In [ ]:
pop_df.head()

In [ ]:
# Clean up the population data
pop_df2 = pop_df[['TIME', '2020', '2021', '2022', '2023']]

# Drop the first row    
pop_df2 = pop_df2.drop(0)

# Rename the first column to 'Country'
pop_df2.rename(columns={'TIME': 'Country'}, inplace=True)

pop_df2.head()

### Estimate death rates in 2020, 2021, 2022 and 2023 for EU countries

In [ ]:
covid_death_df_agg.head()

In [ ]:
# Rename the first column to 'Country'
covid_death_df_agg.rename(columns={'Country/Region': 'Country'}, inplace=True) 

covid_death_df_agg.head()

In [ ]:
covid_death_df_agg['Year'] = covid_death_df_agg['Year'].astype(int)

In [ ]:
# Make wide table long - pop_df2
pop_df2_long = pop_df2.melt(id_vars='Country', var_name='Year', value_name='Population') 

In [ ]:
pop_df2_long.head() 

In [ ]:
pop_df2_long['Year'] = pop_df2_long['Year'].astype(int) 

In [ ]:
# Merge covid_death_df_agg to pop_df2_long on Country and Year columns
covid_death_pop_df = pop_df2_long.merge(covid_death_df_agg, on=['Country', 'Year'], how='left') 


In [ ]:
covid_death_pop_df.head()

In [ ]:
# Create death rate column by dividing Deaths by Population
covid_death_pop_df['Deaths'] = pd.to_numeric(covid_death_pop_df['Deaths'], errors='coerce')
covid_death_pop_df['Population'] = pd.to_numeric(covid_death_pop_df['Population'], errors='coerce')

# Fill NaN values with 0 to avoid division errors
covid_death_pop_df['Deaths'].fillna(0, inplace=True)
covid_death_pop_df['Population'].fillna(0, inplace=True)

# Calculate death rate
covid_death_pop_df['Death Rate'] = covid_death_pop_df['Deaths'] / covid_death_pop_df['Population']

In [ ]:
covid_death_pop_df.head()

### Fermented vegetable consumption data

In [ ]:
# Read in fermented vegetable consumption data in xlsx format
food_df = pd.read_excel('Foodex 2 L4 dashboard.xlsx', skiprows=2)

In [ ]:
food_df.head()

In [ ]:
# Investigate countries, years, and population columns
food_df.rename(columns={"Survey's country": 'Country'}, inplace=True) 
food_df['Country'].value_counts()

In [ ]:
# Survey start year 
food_df['Survey start year'].value_counts()

In [ ]:
# Population 
food_df['Population Group (L2)'].value_counts()

Aggregated daily consumption of fermented vegetables in general population and over time by country.

In [ ]:
avg_consumption_country = food_df.groupby(by ='Country')['Mean'].mean().reset_index()

avg_consumption_country.rename(columns={'Mean': 'Average Consumption'}, inplace=True)

In [ ]:
avg_consumption_country.describe() # Summary statistics 

In [ ]:
avg_consumption_country.head()

Prepare for geographical data of EU countries.

In [ ]:
# Fetch GeoJSON for Europe
import requests
import json
    

In [ ]:
# URL for countries' GeoJSON data
url = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"

# Fetch the data
response = requests.get(url)
geojson_data = response.json()

In [ ]:
# Filter only EU countries
# eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']

In [ ]:
targeted_countries = food_df['Country'].unique().tolist()
print(len(targeted_countries))

In [ ]:
# Filter the geojson for EU 
eu_geojson = {
    "type": "FeatureCollection",
    "features": [
        feature for feature in geojson_data["features"]
        if feature["properties"]["ADMIN"] in targeted_countries
    ]
}

In [ ]:
len(eu_geojson['features'])

## Data frame for the interactive map

In [ ]:
# Identify EU countries in the eu_geojson
eu_countries_geojson = [feature['properties']['ADMIN'] for feature in eu_geojson['features']] 

In [ ]:
print(eu_countries_geojson)

In [ ]:
# Save en_countries_geojson as dataframe 
eu_countries_df = pd.DataFrame(eu_countries_geojson, columns=['Country']) 

In [ ]:
# Merge eu_countries_df with avg_consumption_country on Country
eu_avg_consumption_country = eu_countries_df.merge(avg_consumption_country, on='Country', how='left') 

In [ ]:
eu_avg_consumption_country.head()

In [ ]:
# Merge eu_avg_consumption_country with covid_death_pop_df on Country
eu_avg_consumption_covid_death_pop_df = eu_avg_consumption_country.merge(covid_death_pop_df, on='Country', how='left')

In [ ]:
eu_avg_consumption_covid_death_pop_df.columns   

In [ ]:
# # Draw a scatter plot of Average Consumption vs Death Rate, stratified by Year
# plt.figure(figsize=(12, 6))

# # Add line of best fit by year
# sns.lmplot(data=eu_avg_consumption_covid_death_pop_df, x='Average Consumption', y='Death Rate', hue='Year', palette='viridis', height=6, aspect=2)
# plt.title('Average Consumption vs Death Rate by Year')

# plt.show()

## Bubble map of fermented vegetable consumption in EU countries

To change the colors of the areas (countries) separately from the bubbles, use px.choropleth() for the country colors and px.scatter_geo() for the bubbles, then overlay them using go.Figure().

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [ ]:
eu_avg_consumption_covid_death_pop_df.columns 

In [ ]:
eu_avg_consumption_covid_death_pop_df.head()

In [ ]:
# Drop rows with Inf values in Death Rate column
eu_avg_consumption_covid_death_pop_df = eu_avg_consumption_covid_death_pop_df[eu_avg_consumption_covid_death_pop_df['Death Rate'] != np.inf]

In [ ]:
import pycountry

# List of EU countries
eu_countries = eu_avg_consumption_covid_death_pop_df['Country'].unique().tolist()   

# Dictionary of country names and their corresponding alpha_3 codes
country_alpha3 = {}
for country in eu_countries:
    try:
        country_data = pycountry.countries.get(name=country)
        # print(country_data.alpha_3)
        country_alpha3[country] = country_data.alpha_3
    except:
        print(f"{country} not found")

print(country_alpha3)


In [ ]:
# Add ISO Alpha-3 codes to eu_avg_consumption_covid_death_pop_df
eu_avg_consumption_covid_death_pop_df['iso_alpha'] = eu_avg_consumption_covid_death_pop_df['Country'].map(country_alpha3)


In [ ]:
data_map_2020 = eu_avg_consumption_covid_death_pop_df[eu_avg_consumption_covid_death_pop_df['Year'] == 2020]

In [ ]:
# Create a Choropleth map (for country colors) based on fermented vegetable consumption
food_map = px.choropleth(
    data_map_2020,
    locations="iso_alpha",
    color="Average Consumption",
    hover_name="Country",
    scope="europe",
    projection="natural earth",
    color_continuous_scale='Plasma'
)

food_map.show()


In [ ]:
data_map_2020.head()

In [ ]:

# df = px.data.gapminder().query("year==2007")
bubble_map = px.scatter_geo(data_map_2020,
                            locations="iso_alpha",
                            hover_name="Country",
                            size="Death Rate",
                            scope="europe",
                            projection="natural earth",
                            opacity=0.7, # Set opacity level for better visibility
                            size_max=15,
                            color_continuous_scale=px.colors.sequential.Plasma)
bubble_map.show()

In [ ]:
# Combine both layers
fig = go.Figure(data=food_map.data + bubble_map.data)

In [ ]:
# Improve layout
fig.update_geos(
    scope="europe", # Only show European countries
    showcoastlines=False, 
    showland=True, 
    landcolor="lightgray",
    projection_scale=1.5)

fig.update_layout(
    coloraxis_colorbar_title="Fermented Vegetable Consumption",
    coloraxis_colorscale="RdYlBu" , # Change color scale
    width=1200,
    height=800,
    coloraxis_colorbar=dict(
        orientation="h",  # Set colorbar horizontal
        title="Fermented Vegetable Consumption",
        title_side="top",
        title_font_size=12,
        thickness=10,  # Adjust colorbar width
        len=0.5,  # Adjust colorbar height (relative size)
        x=0.25,  # Move colorbar horizontally
        y=0.95,  # Move colorbar vertically
    )
)

fig.update_layout(
    title=dict(
        text="Fermented Vegetable Consumption and Covid-19 Death Rate in Europe (2020)",
        x=0.5,  # Center the title
        y=0.98,  # Position it above the colorbar
        xanchor="center",  # Ensure proper centering
        yanchor="top",  # Anchor at the top
        font=dict(
            size=18,  # Increase font size for better readability
            family="Arial, sans-serif",  # Use a professional font
            color="black",  # Set color (adjust if needed)
            weight="bold"  # Bolden the title (alternative: use "<b>Title</b>" in text)
        )
    )
)

fig.update_layout(
    coloraxis_colorbar=dict(
        orientation="h",  # Horizontal colorbar
        x=0.5, y=-0.15,  # Move below the map
        len=0.5, thickness=10
    )
)

fig.show()

In [ ]:
# Annotate country names on the map

import pandas as pd

# Create the DataFrame
country_data = pd.DataFrame({
    "Country": ["Austria", "Belgium", "Bulgaria", "Bosnia and Herzegovina", "Germany", "Estonia", "Finland", "France", "United Kingdom", "Greece", "Croatia", "Hungary", "Latvia", "Montenegro", "Netherlands", "Poland", "Portugal", "Romania", "Slovenia", "Sweden"],
    "ISO3": ["AUT", "BEL", "BGR", "BIH", "DEU", "EST", "FIN", "FRA", "GBR", "GRC", "HRV", "HUN", "LVA", "MNE", "NLD", "POL", "PRT", "ROU", "SVN", "SWE"],
    "Lat": [47.5162, 50.5039, 42.7339, 43.9159, 51.1657, 58.5953, 61.9241, 46.6034, 55.3781, 39.0742, 45.1, 47.1625, 56.8796, 42.7087, 52.1326, 51.9194, 39.3999, 45.9432, 46.1512, 60.1282],
    "Lon": [14.5501, 4.4699, 25.4858, 17.6791, 10.4515, 25.0136, 25.7482, 1.8883, -3.4360, 21.8243, 15.2, 19.5033, 24.6032, 19.3744, 5.2913, 19.1451, -8.2245, 24.9668, 14.9955, 18.6435]
})

# Display the DataFrame
print(country_data)

In [ ]:
import plotly.graph_objects as go

# Create the country label layer (scattergeo)
country_labels = go.Scattergeo(
    locationmode="ISO-3",
    lon=country_data["Lon"],
    lat=country_data["Lat"],
    text=country_data["Country"],  # Display country names
    mode="text",  # Only text (no markers)
    textfont=dict(size=12, color="black", family="Arial", weight="bold"),  # Adjust font
    textposition="top center",  
    showlegend=False
)

# Add to your existing Plotly figure
fig.add_trace(country_labels)


In [ ]:
# Add footnote 
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=200)  # Increase bottom margin (b) for the footnote
)  

fig.add_annotation(
    text="Data source: The population data, Covid-19 mortality, and fermented food consumption data is from Eurostat, Johns Hopkins Coronavirus Resource Center,<br>"
    "and European Food Safety Authority (EFSA) Comprehensive European Food Consumption Database, respectively.<br>"
    "The map is inspired by the preprint, Association between consumption of fermented vegetables and COVID-19 mortality at a country level in Europe, by Fonseca et al. (2020).",
    xref="paper", yref="paper",
    x=0.5, y=-0.3,
    showarrow=False, # No arrow needed
    font=dict(size=12, color="grey", family="Arial"),
    align="left"
)

fig.show()

In [ ]:
# Save the Plotly figure as HTML file  
pio.write_html(fig, file='index.html', auto_open=True)

In [ ]:
pio.write_image(fig, 'fermented_vegetable_consumption_covid_death_rate_europe_2020.png', format='png', width=1200, height=800, scale=2) # Save as PNG